In [1]:
import pandas as pd
import numpy as np
import csv
import StringIO
import matplotlib.pyplot as plt

# Creating csv file of firms with terms as columns

In [2]:
## Read file in AFTER CLEANED on 180

df = pd.read_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/TermResults_QA_Acquisition.csv')

In [3]:
##df=df.convert_objects(convert_numeric=True, convert_dates=True)
df.stockSymbol.value_counts()

wmgi    87
xrx     82
bbt     80
syk     80
orcl    79
sbux    78
jci     78
rrd     78
t       77
cvs     77
wm      77
pwr     77
csco    76
col     76
tyc     76
...
upip    1
cers    1
mxc     1
cybx    1
pfsi    1
ig      1
kool    1
ccel    1
sm      1
okme    1
petv    1
cvrs    1
opvs    1
fcel    1
agoe    1
Length: 4072, dtype: int64

In [16]:
pd.set_option('display.max_rows', 1000)
df.termName.value_counts()

Acquisition                                          39032
Cash Paid for Acquisition                            34276
Acquisition, Equity Interests Issued and Issuable     1426
dtype: int64

In [18]:
## This transposes termnames into Columns

acq =df[df.termName=="Acquisition"][["stockSymbol","FY","FQ","value"]]
acq1 = df[df.termName=="Cash Paid for Acquisition"][["stockSymbol","FY","FQ","value"]]
acq2 = df[df.termName=="Acquisition, Equity Interests Issued and Issuable"][["stockSymbol","FY","FQ", "value"]]


acq.columns = ["stockSymbol", "FY", "FQ", "Acquisition"]
acq1.columns = ["stockSymbol", "FY", "FQ", "Cash Paid for Acquisition"]
acq2.columns = ["stockSymbol", "FY", "FQ", "Acquisition, Equity Interests Issued and Issuable"]

joined = pd.merge(acq,acq1, on=["stockSymbol", "FY", "FQ"])
joined = pd.merge(joined, acq2, on=["stockSymbol", "FY", "FQ"])

#joined=joined.convert_objects(convert_numeric=True)
#joined['nini']=joined['nini'].dropna()
#joined['nini']=joined[joined['nini'] !=0][['nini']]

#joined["bad"] = joined.apply(lambda x: x.fcfpni != float("{0:.2f}".format(x.fcf / x.nini)), axis=1)
#joined["badcalc"] = joined.apply(lambda x: float("{0:.2f}".format(x.fcf / x.nini)), axis=1)

# some values are NaN so we use notnull()
#bad = joined[joined.bad][joined.fcf.notnull()]

#joined=joined.convert_objects(convert_numeric=True)
#joined['nini']=joined['nini'].dropna()
#joined['nini']=joined[joined['nini'] !=0][['nini']]

joined.to_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/TermResults_QA_Acquisition_reshape.csv')

# Creating time-series of terms for companies resolving terms

In [27]:

## Generating term coverage by stockSymbol, FY and FQ - Time Series
df['counter']=1
table = pd.pivot_table(df, values='counter', rows=['termName', 'stockSymbol','expression'],
                    cols=['FY', 'FQ'], aggfunc=np.sum)

table['total']=table.sum(axis=1,skipna=True)  ##  Adding across the columns in the dataframe and skipping NAs
table.to_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/Table_TermResults_QA_Sherry.csv')

# Creating time-series of all terms for all companies 

In [28]:
## Read in list of all companies from EntityReferences. This is the universe of all companies in our database.

ss=pd.read_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/EntityReferences.csv')
ss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7249 entries, 0 to 7248
Data columns (total 3 columns):
entityId       7249 non-null int64
cik            7249 non-null object
stockSymbol    7222 non-null object
dtypes: int64(1), object(2)
memory usage: 226.5+ KB


In [29]:
ss.stockSymbol.value_counts()

fhlb    12
mep      9
rexi     6
xel      5
pru      4
met      4
voya     3
nyrt     3
cqp      3
ppl      3
tln      3
spb      3
sum      3
prty     3
wmb      3
...
trns     1
adtm     1
hall     1
nor      1
haln     1
uhn      1
name     1
kerx     1
namg     1
ar       1
gdsi     1
gryn     1
arwau    1
clwd     1
blgo     1
Length: 7034, dtype: int64

In [32]:
## Read in List of terms mongexported
terms=pd.read_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/Term_QA_Sherry.csv', index_col=False, header=0)
terms.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 1 columns):
termName    4 non-null object
dtypes: object(1)
memory usage: 64.0+ bytes


In [33]:
terms.termName.value_counts()

Trading Securities, Current                 1
Available for Sales Securities, Current,    1
Held To Maturity Securities, Current,       1
Marketable Securities, Current,             1
dtype: int64

In [34]:
## Note, the zip function does not iterate through each element in the list. Just does a pairwise combination based on element location.


t=terms['termName']
s=ss['stockSymbol']

In [35]:
## This is the more appropriate way to combine two lists of unqeual length to get the pairwise combination.  Insert the | to bifrucate the two strings

import itertools
c=map (lambda (x,y):str(x)+ '|' + str(y), list (itertools.product (t, s)))

In [36]:
##Now, convert c into a dataframe
c=pd.DataFrame(c, columns=['c'])
c.head()

,c
0,"Available for Sales Securities, Current,|pson"
1,"Available for Sales Securities, Current,|vnrx"
2,"Available for Sales Securities, Current,|fonu"
3,"Available for Sales Securities, Current,|airi"
4,"Available for Sales Securities, Current,|soul"


In [37]:
## Now, need tosplit the column based on |

splits = c['c'].str.split('|')
c['termName'] = splits.str[0]
c['stockSymbol'] = splits.str[1]

c.head()

,c,termName,stockSymbol
0,"Available for Sales Securities, Current,|pson","Available for Sales Securities, Current,",pson
1,"Available for Sales Securities, Current,|vnrx","Available for Sales Securities, Current,",vnrx
2,"Available for Sales Securities, Current,|fonu","Available for Sales Securities, Current,",fonu
3,"Available for Sales Securities, Current,|airi","Available for Sales Securities, Current,",airi
4,"Available for Sales Securities, Current,|soul","Available for Sales Securities, Current,",soul


In [38]:
##Create empty columns or nan 
##c['2009Q1']="" / Create empty column
## c['2009Q1]-np.nan / Creat columns filled with nan

c['2009Q1']=np.nan
c['2009Q2']=np.nan
c['2009Q3']=np.nan
c['2009Q4']=np.nan
c['2009FY']=np.nan
c['2010Q1']=np.nan
c['2010Q2']=np.nan
c['2010Q3']=np.nan
c['2010Q4']=np.nan
c['2010FY']=np.nan
c['2011Q1']=np.nan
c['2011Q2']=np.nan
c['2011Q3']=np.nan
c['2011Q4']=np.nan
c['2011FY']=np.nan
c['2012Q1']=np.nan
c['2012Q2']=np.nan
c['2012Q3']=np.nan
c['2012Q4']=np.nan
c['2012FY']=np.nan
c['2013Q1']=np.nan
c['2013Q2']=np.nan
c['2013Q3']=np.nan
c['2013Q4']=np.nan
c['2013FY']=np.nan
c['2014Q1']=np.nan
c['2014Q2']=np.nan
c['2014Q3']=np.nan
c['2014Q4']=np.nan
c['2014FY']=np.nan
c['2015Q1']=np.nan
c['2015Q2']=np.nan

In [39]:
## Need to drop c['c'] which is the combination of termName and stockSymbol to merge.
d=c.drop('c', axis=1)
d.termName.value_counts()

Marketable Securities, Current,             7249
Available for Sales Securities, Current,    7249
Held To Maturity Securities, Current,       7249
Trading Securities, Current                 7249
dtype: int64

In [40]:
## Save full empty cartesian

d.to_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/EmptyCartesian_QA_Sherry.csv')

In [41]:
## Read back in EmptyCartesian to test
d = pd.read_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/EmptyCartesian_QA_Sherry.csv')

In [42]:
##Read in term results (after cleaning the table csv file)


df=pd.read_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/Table_TermResults_QA_Sherry.csv')

In [43]:
df

,termName,stockSymbol,expression,2010FY,2010Q1,2010Q2,2010Q3,2010Q4,2011FY,2011Q1,...,2014Q3,2014Q4,2015FY,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,total
0,"Available for Sales Securities, Current",a,nameContains(AvailableForSale) && nameNotConta...,1,NaN,NaN,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
1,"Available for Sales Securities, Current",aame,nameContains(AvailableForSale) && nameNotConta...,NaN,NaN,NaN,NaN,NaN,1,NaN,...,1,1,NaN,1,1,1,NaN,NaN,NaN,19
2,"Available for Sales Securities, Current",aapl,{us-gaap_AvailableForSaleSecuritiesCurrent},1,1,1,NaN,1,1,NaN,...,1,1,1,1,1,1,1,1,NaN,33
3,"Available for Sales Securities, Current",aapl,{us-gaap_AvailableForSaleSecuritiesDebtSecurit...,NaN,NaN,NaN,1,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,"Available for Sales Securities, Current",aavl,{us-gaap_AvailableForSaleSecuritiesCurrent},NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,2
5,"Available for Sales Securities, Current",abax,nameContains(AvailableForSale) && nameNotConta...,NaN,NaN,NaN,NaN,NaN,1,NaN,...,1,NaN,NaN,1,1,1,NaN,NaN,NaN,13
6,"Available for Sales Securities, Current",abcb,nameContains(AvailableForSale) && nameNotConta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
7,"Available for Sales Securities, Current",abco,{us-gaap_AvailableForSaleSecuritiesDebtSecurit...,1,NaN,NaN,NaN,1,1,NaN,...,1,NaN,NaN,1,1,NaN,NaN,NaN,NaN,20
8,"Available for Sales Securities, Current",abcw,{us-gaap_AvailableForSaleSecuritiesDebtSecurit...,NaN,NaN,NaN,NaN,NaN,1,NaN,...,1,1,NaN,1,1,1,NaN,NaN,NaN,19
9,"Available for Sales Securities, Current",abm,nameContains(AvailableForSale) && nameNotConta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,NaN,NaN,NaN,1,NaN,NaN,12


In [44]:
## Merge term results with empty cartesian file - NOTE, do an outer join to keep the union of the two data frames, do not specific the coumns to merge on.

c = pd.merge(d, df, how='outer')


In [159]:
## Drop empty row where total is null
##c = c[pd.notnull(c["Unnamed: 38"])]


In [45]:
c.to_csv('/Users/christine/Documents/idaciti/QAFramework/TermResults/Table_TermResults_QA_Sherry_Clean.csv')

# Testing Coverage - Merging FYE, earliest and latest filing dates CORRECT


In [26]:
merged_final=pd.read_csv('/Users/christine/Documents/idaciti/QAFramework/Table_TermResults_80Percent_ForQA_Ryan_1.csv')

In [27]:
FYE=pd.read_csv('/Users/christine/Documents/idaciti/QAFramework/SECFilings_10_30_2015.csv')

In [28]:
FYE.info()
FYE=FYE.sort(['stockSymbol','filingDate'], ascending=[1,1]) ## Sort SECFilings by stockSymbol and filingDate
FYE.stockSymbol.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87216 entries, 0 to 87215
Data columns (total 2 columns):
stockSymbol    87214 non-null object
filingDate     87214 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.0+ MB


afh     129
eme      55
duk      43
emc      42
wec      42
eix      40
arct     35
esv      35
alex     35
alco     34
aiv      34
fbp      34
pfsi     34
cmxi     34
pcl      33
...
blx     4
arco    4
gigm    4
ncty    4
feng    4
rada    4
ambo    4
csiq    4
cste    3
eca     3
trp     3
yy      3
cnit    3
zayo    2
soho    1
Length: 4972, dtype: int64

In [29]:
fye=FYE[['stockSymbol', 'filingDate']]

In [30]:
fye_latest=fye.drop_duplicates(subset=['stockSymbol'], take_last=True)  ## Take the latest filingDate, otherwise defaults to first row

In [31]:
fye_earliest=fye.drop_duplicates(subset=['stockSymbol'])

In [32]:
fye_earliest

,stockSymbol,filingDate
35058,a,20091005
47272,aa,20090410
84504,aamc,20130509
35254,aame,20110812
14906,aan,20100804
19779,aaoi,20131114
20420,aaon,20110804
44416,aap,20100602
47284,aapl,20090722
69608,aasp,20110812


In [33]:
c = pd.merge(merged_final, fye_earliest, on='stockSymbol')

In [34]:
c=pd.merge(c,fye_latest, on='stockSymbol')

In [35]:
c.head()

,Unnamed: 0,c,termName,stockSymbol,2009Q1,2009Q2,2009Q3,2009Q4,2009FY,2010Q1,...,2014FY,2015Q1,2015Q2,elementName,2015FY,2015Q3,2015Q4,Unnamed: 38,filingDate_x,filingDate_y
0,0,NaN,Current Income Tax Expense (Benefit),airi,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,us-gaap_CurrentIncomeTaxExpenseBenefit,NaN,NaN,NaN,8,20121002,20150812
1,1,Current Income Tax Expense (Benefit)|airi,Current Income Tax Expense (Benefit),airi,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20121002,20150812
2,8232,NaN,"Current Income Tax Expense (Benefit), Federal",airi,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,us-gaap_CurrentFederalTaxExpenseBenefit,NaN,NaN,NaN,22,20121002,20150812
3,8233,"Current Income Tax Expense (Benefit), Federal|...","Current Income Tax Expense (Benefit), Federal",airi,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20121002,20150812
4,16414,"Current Income Tax Expense (Benefit), Foreign|...","Current Income Tax Expense (Benefit), Foreign",airi,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20121002,20150812


In [36]:
c['total']=c.sum(axis=1,skipna=True)

##First, need to sort dataframe and place NaN at the end
merged_final=c.sort(columns=['stockSymbol', 'termName'], na_position='last')

## So, when drop duplicates, will drop last total (NaN)
merged_final=merged_final.drop_duplicates(subset=['termName', 'stockSymbol'])

In [37]:
merged_final.to_csv('/Users/christine/Documents/idaciti/QAFramework/Table_TermResults_80Percent_ForQA_Ryan_1_with FYE.csv')

In [39]:
merged_final.stockSymbol.value_counts()

bcr      12
nbr      12
carb     12
sbgi     12
argb     12
orb      12
orc      12
gps      12
ora      12
tpoi     12
utsi     12
turv     12
czfs     12
chscp    12
gtim     12
...
rox     12
audc    12
xtrn    12
rig     12
engt    12
olbg    12
trst    12
feim    12
hsii    12
glgi    12
vsh     12
olbk    12
wwww    12
blgo    12
zayo    11
Length: 4972, dtype: int64